In [3]:
# !pip install google-genai

# JSON

In [4]:
import json

# Load the JSON file
json_file_path = "metadata/paglia_metadata.json"

with open(json_file_path, "r") as file:
    pagila_metadata = json.load(file)

# Display the structure of the JSON file (showing only a subset if it's large)
pagila_metadata


{'database_name': 'paglia',
 'description': 'A database for managing a DVD rental store.',
 'tables': [{'table_name': 'actor',
   'description': 'Stores information about actors.',
   'columns': [{'column_name': 'actor_id',
     'data_type': 'integer',
     'is_primary_key': True,
     'description': 'Unique identifier for the actor.'},
    {'column_name': 'first_name',
     'data_type': 'text',
     'description': "Actor's first name."},
    {'column_name': 'last_name',
     'data_type': 'text',
     'description': "Actor's last name."},
    {'column_name': 'last_update',
     'data_type': 'timestamp with time zone',
     'description': 'Timestamp of the last update to the record.'}]},
  {'table_name': 'address',
   'description': 'Stores address information.',
   'columns': [{'column_name': 'address_id',
     'data_type': 'integer',
     'is_primary_key': True,
     'description': 'Unique identifier for the address.'},
    {'column_name': 'address',
     'data_type': 'text',
     'de

In [9]:
names = list(map(str, list(pagila_metadata.keys())))
names

['database_name',
 'description',
 'tables',
 'views',
 'materialized_views',
 'functions',
 'relationships']

In [23]:
metadata = json.dumps(pagila_metadata)
len(metadata)

23036